In [2]:
from Utils.loadset import getDataSet
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import tensorflow_probability as tfp


tfk = tf.keras
tfkl = tf.keras.layers
tfpl = tfp.layers
tfd = tfp.distributions
import os
from trainer import Trainer
try:
    from Utils.connection_cfg import *
except Exception as e:
    PSWD = None
    USRN = None
    
from Utils.Data import dataWrapper
from Utils.transform import ToCategorical, cutOut

Num GPUs Available: 1


Using TensorFlow backend.


In [3]:
dimension = (64,64)
batch_size = 10
channels = 5
optimizer = Adam( lr = 1e-6 )
slices = [256,320,256,320]
cutOutFrame = cutOut(slices)

categorical_list = [0,1,5,10,15,30,60,120]
categorical = ToCategorical(categorical_list)

PRETRAINING_TRANSFORMATIONS = [cutOutFrame]
TRANSFORMATIONS = [categorical]
TRANSFORMATIONS = None

def NLL(y_true, y_hat):
    return -y_hat.log_prob(y_true)


def provideData(flatten=False,dimension=dimension,batch_size=60,transform=None,preTransformation=None):

    getDataSet(DatasetFolder,year=[2017],username=USRN,pswd=PSWD)
    train,test = dataWrapper(PathToData,
                            dimension=dimension,
                            channels=channels,
                            batch_size=batch_size,
                            overwritecsv=True,
                            flatten=flatten,
                            onlyUseYears=[2017],
                            transform=transform,
                            preTransformation=preTransformation)
    
    return train,test
DatasetFolder = "./Data/RAW"
PathToData = os.path.join(DatasetFolder,"MonthPNGData")

train, test = provideData(dimension=dimension,
                          batch_size=batch_size,
                          transform=TRANSFORMATIONS,
                          preTransformation=PRETRAINING_TRANSFORMATIONS)

Found Year :  2017 => won't download this year again... please check for consistency
Finished Loading Dataset
 


In [11]:
y = train[0][1]
print(y.shape)
cat = tfp.distributions.Categorical(probs=tf.zeros_like(tf.expand_dims(y,axis=-1) ))
print(tf.stack([1-y[:,:,:,0], y[:,:,:,0]],axis=-1).shape)
cat = tfp.distributions.Categorical(probs=tf.stack([1-y, y],axis=-1))
pos = tfp.distributions.Poisson(rate=y)
det = tfp.distributions.Deterministic(loc=y)

print(cat)
print(det)
print(pos)
components = [det,pos]
mix = tfd.Mixture(cat = cat,components=components)
print(mix)

(10, 64, 64, 1)
(10, 64, 64, 2)
tfp.distributions.Categorical("Categorical", batch_shape=[10, 64, 64, 1], event_shape=[], dtype=int32)
tfp.distributions.Deterministic("Deterministic", batch_shape=[10, 64, 64, 1], event_shape=[], dtype=float64)
tfp.distributions.Poisson("Poisson", batch_shape=[10, 64, 64, 1], event_shape=[], dtype=float64)
tfp.distributions.Mixture("Mixture", batch_shape=[10, 64, 64, 1], event_shape=[], dtype=float64)


In [16]:
print(cat)
print(pos)
print(det)

tfp.distributions.Categorical("Categorical", batch_shape=[1, 64, 64], event_shape=[], dtype=int32)
tfp.distributions.Poisson("Poisson", batch_shape=[1, 64, 64, 1], event_shape=[], dtype=float64)
tfp.distributions.Deterministic("Deterministic", batch_shape=[1, 64, 64, 1], event_shape=[], dtype=float64)
